# Spiky / Discretized Numeric Features（离散尖刺）处理备忘

**场景**：某些“数值型”特征看起来是 `float/int`，但分布呈现明显 **spikes**（少数取值占据大量样本），常见成因：**rounding / bucket / tick size / 报表口径离散化**。

**核心结论**：这类变量不再满足“连续可微/平滑”的直觉；将其当作纯连续变量会诱导模型做不合理的插值/外推。处理时优先考虑 **categorical / ordinal / target encoding / 树模型** 等更贴近生成机制的方法。

本 notebook 作为 dataset interview 的 quick reference：
- 如何识别 spiky
- 如何快速判断该走 categorical/ordinal/target encoding
- 如何在时序切分下避免泄漏
- 如何在 presentation 里用一句话解释处理逻辑

## 0. 环境与工具

默认使用：`numpy, pandas, scikit-learn, matplotlib`（面试环境通常具备）。  
如需额外包（例如 `category_encoders`），用 `pip install`（现场以可用性为准）。

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

## 1) Spiky 的快速体征（5分钟 checklist）

一个变量满足任意多条，即可判定为“高度离散/尖刺”：
- `n_unique` 很小（例如 < 50 / < 100），但样本量很大
- `value_counts()` 前几个取值占比离谱（例如 top-5 占比 > 50%）
- 小数位“锁死”（例如全是 0.01 的倍数、全是整数、全是 0.5 的倍数）
- 直方图呈现“针状”（mass 集中在少数网格点）
- 时序上表现为：长时间不动，偶尔跳一下（报价 tick / bucket 更新）

下面的函数输出：
- unique 数、top-k 占比、最常见取值
- 小数粒度（近似）
- 分布图（直方图 + 取值条形图）

In [ ]:

def _decimal_step_estimate(x, max_n=5000):
    """
    Estimate discretization step (heuristic).
    Method: sample -> unique -> sort -> diffs -> mode of positive diffs.
    """
    x = pd.Series(x).dropna().astype(float)
    if x.empty:
        return np.nan
    if len(x) > max_n:
        x = x.sample(max_n, random_state=0)
    vals = np.sort(x.unique())
    if len(vals) < 3:
        return np.nan
    diffs = np.diff(vals)
    diffs = diffs[np.isfinite(diffs) & (diffs > 0)]
    if len(diffs) == 0:
        return np.nan
    diffs_r = np.round(diffs, 12)
    step = pd.Series(diffs_r).value_counts().idxmax()
    return float(step)

def spiky_report(df, col, topk=10, bins=50, show_plots=True):
    s = df[col]
    n = len(s)
    nunique = s.nunique(dropna=True)
    vc = s.value_counts(dropna=True)
    top_share = vc.head(topk).sum() / vc.sum() if vc.sum() > 0 else np.nan
    step = _decimal_step_estimate(s)

    print(f"[{col}] n={n:,}  nunique={nunique:,}  top{topk}_share={top_share:.3f}  est_step={step}")
    print("Top values:")
    display(vc.head(topk))

    if not show_plots:
        return

    # Histogram
    plt.figure()
    s_float = pd.to_numeric(s, errors="coerce")
    plt.hist(s_float.dropna().values, bins=bins)
    plt.title(f"{col} histogram")
    plt.xlabel(col)
    plt.ylabel("count")
    plt.show()

    # Bar plot when nunique is small
    if nunique <= 60:
        plt.figure()
        vc.head(min(30, nunique)).sort_index().plot(kind="bar")
        plt.title(f"{col} top values barplot (sorted by value)")
        plt.xlabel(col)
        plt.ylabel("count")
        plt.show()

### 1.1 演示（用模拟数据）

下面造几种典型 spiky：
- rounding：连续值四舍五入到 0.05
- bucket：先分箱再映射为箱中心/等级
- 混合：大量 0 + 少量连续值（“零尖刺”）

In [ ]:

rng = np.random.default_rng(0)
n = 20000

df_demo = pd.DataFrame({
    "x_round_0p05": np.round(rng.normal(0, 1, n) / 0.05) * 0.05,
    "x_bucket_10": pd.cut(rng.normal(0, 1, n), bins=10, labels=False).astype(float),
    "x_zero_spike": (rng.random(n) < 0.7).astype(float) * 0.0 + (rng.random(n) >= 0.7) * rng.normal(0, 1, n),
})
df_demo["y_reg"] = 0.5*df_demo["x_round_0p05"] + 0.2*df_demo["x_bucket_10"] + rng.normal(0, 0.5, n)

for c in ["x_round_0p05", "x_bucket_10", "x_zero_spike"]:
    spiky_report(df_demo, c, topk=10, bins=60)

## 2) 为什么“连续假设”会失效

spiky 数值特征的风险点（可用于解释/写在结论页）：
1. 变量并非真正连续，距离/梯度意义弱；线性/核方法容易做不合理插值  
2. 分布强离散会导致残差模式异常（异方差、分段结构）  
3. 取值重复率高时，编码（尤其 target encoding）在不恰当切分下容易“键值记忆”导致泄漏/过拟合  
4. 树模型对阈值切分天然匹配此类变量，更稳健

因此：优先把它当作 **类别/序数** 或交给 **树模型**；若使用 target encoding，必须严格 OOF（时序用 past-only）。

## 3) 处理路线图（选择逻辑）

把一个 spiky 数值特征 `x` 分成三类场景：

### A. 取值很少（例如 nunique < 50）
- 直接当作 categorical（OneHot）
- 或明确有顺序含义时当作 ordinal（保留整数等级）

### B. 取值中等偏多，但仍强离散（例如 50 ~ 5k）
- OneHot 维度可能太大
- 用 target encoding（OOF + smoothing）
- 或保留为数值但切换为树模型

### C. 取值非常多（接近连续）
- 这通常已经不是 spike，而是连续噪声
- 走常规数值处理即可（标准化、winsorize、对数等）

## 4) Target Encoding（OOF + smoothing）模板

直接用全量 `y` 对每个取值算均值再编码会泄漏标签信息（重复率高时相当于“查表”）。  
OOF 做法：对每一折，用其余折的统计量去编码当前折。

平滑（smoothing）用于小样本取值，避免极端均值：  
`enc(v) = (n_v*mean_v + alpha*global_mean) / (n_v + alpha)`

In [ ]:

def target_encode_oof(x, y, n_splits=5, alpha=20.0, random_state=0):
    """
    OOF target encoding with smoothing (regression).
    x: categorical-like feature (can be numeric but spiky)
    y: target
    returns: encoded series aligned with x.index
    """
    x = pd.Series(x)
    y = pd.Series(y)
    global_mean = y.mean()

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    enc = pd.Series(index=x.index, dtype=float)

    for tr_idx, va_idx in kf.split(x):
        x_tr, y_tr = x.iloc[tr_idx], y.iloc[tr_idx]
        stats = y_tr.groupby(x_tr).agg(["mean", "count"])
        smooth = (stats["count"] * stats["mean"] + alpha * global_mean) / (stats["count"] + alpha)
        enc.iloc[va_idx] = x.iloc[va_idx].map(smooth).fillna(global_mean).astype(float)

    return enc

enc_demo = target_encode_oof(df_demo["x_bucket_10"], df_demo["y_reg"], n_splits=5, alpha=50.0)
enc_demo.head()

## 5) 时间序列场景：past-only target encoding（避免未来信息）

时间序列下 encoding 必须只使用“过去”：
- 对每个时间点 t 的样本，只能用 < t 的数据估计映射  
- 常用：expanding mean / rolling mean + smoothing

下面是最简 past-only 实现（逐行更新）。

In [ ]:

from collections import defaultdict

def target_encode_past_only(x, y, time, alpha=20.0):
    """
    Past-only target encoding with smoothing (regression).
    encoding at row i uses only rows with time < time_i.
    """
    df = pd.DataFrame({"x": x, "y": y, "t": time}).sort_values("t")
    global_sum = 0.0
    global_cnt = 0

    sum_by = defaultdict(float)
    cnt_by = defaultdict(int)

    enc_sorted = np.empty(len(df), dtype=float)

    for i, (xi, yi) in enumerate(zip(df["x"].values, df["y"].values)):
        global_mean = (global_sum / global_cnt) if global_cnt > 0 else 0.0

        c = cnt_by.get(xi, 0)
        s = sum_by.get(xi, 0.0)
        if c == 0:
            enc_val = global_mean
        else:
            mean_v = s / c
            enc_val = (c * mean_v + alpha * global_mean) / (c + alpha)

        enc_sorted[i] = enc_val

        global_sum += float(yi)
        global_cnt += 1
        sum_by[xi] += float(yi)
        cnt_by[xi] += 1

    enc = pd.Series(enc_sorted, index=df.index).sort_index()
    return enc.reindex(pd.Index(x.index))

df_demo_ts = df_demo.copy()
df_demo_ts["t"] = np.arange(len(df_demo_ts))

enc_past = target_encode_past_only(df_demo_ts["x_bucket_10"], df_demo_ts["y_reg"], df_demo_ts["t"], alpha=50.0)
enc_past.head()

## 6) categorical / ordinal 处理模板 + baseline 对比

树模型对 spiky 变量更稳健（阈值切分天然匹配离散台阶）。  
线性模型若要吃这类变量，更安全的做法是 OneHot（categorical），而非强行当连续数值。

In [ ]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split

X = df_demo[["x_bucket_10", "x_round_0p05", "x_zero_spike"]]
y = df_demo["y_reg"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

cat_cols = ["x_bucket_10"]
num_cols = ["x_round_0p05", "x_zero_spike"]

pre = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

model_ridge = Pipeline([("pre", pre), ("model", Ridge(alpha=1.0))])
model_tree = HistGradientBoostingRegressor(random_state=0)

model_ridge.fit(X_train, y_train)
model_tree.fit(X_train, y_train)

pred_r = model_ridge.predict(X_test)
pred_t = model_tree.predict(X_test)

rmse_r = mean_squared_error(y_test, pred_r, squared=False)
rmse_t = mean_squared_error(y_test, pred_t, squared=False)

print("RMSE (OneHot+Ridge):", rmse_r)
print("RMSE (Tree baseline):", rmse_t)

## 7) Presentation 

- “`<feature>` is numeric but heavily discretized (spiky distribution), likely due to rounding/bucketing. Treating it as continuous may introduce misleading interpolation, so it’s handled as categorical/ordinal (and compared with OOF target encoding). Tree-based models are naturally robust here and serve as a strong baseline.”

时间序列版本：
- “To avoid leakage, any target encoding for `<feature>` uses past-only statistics (expanding/rolling) consistent with the time split.”

## 8) 现场最小操作顺序（自检）

1. `spiky_report(df, col)`：确认分布形态、取值规模、步长  
2. `nunique` 很小：categorical / ordinal  
3. `nunique` 中等且 OneHot 会爆：OOF target encoding（时序用 past-only）  
4. baseline：树模型（GBDT/HistGB） vs 线性（Ridge/Lasso）  
5. 汇报：强调“生成机制导致离散化”，并说明 encoding 如何避免泄漏